here is this notebook, you learn how to write a local mcp server.<br>
[documentation](https://modelcontextprotocol.io/quickstart/server)<br>
[github](https://github.com/modelcontextprotocol/quickstart-resources/tree/main/weather-server-python)

steps:
- Initalize venv and install dependencies
- Building your server with FastMCP
- write helper functions
- Implementing tool execution: exposing tools to MCP server

### Initalize venv and install dependencies

```bash
# Create a new directory for our project
uv init weather
cd weather

# Create virtual environment and activate it
uv venv
source .venv/bin/activate

# Install dependencies
uv add "mcp[cli]" httpx

# Create our server file
touch weather.py
```

### Building your server with FastMCP

```python
from typing import Any
import httpx
from mcp.server.fastmcp import FastMCP

# Initialize FastMCP server
mcp = FastMCP("weather")

# Constants
NWS_API_BASE = "https://api.weather.gov"
USER_AGENT = "weather-app/1.0"
```

**Q** what is `mcp = FastMCP("weather")`?<br>
A. FastMCP likely refers to a FastAPI-based implementation of a server for Model Context Protocol (MCP). it creates a local MCP server.


### Implementing tool execution: 
exposing tools to MCP server<br>
you expose new tools and add new capabilites to your mcp server.

**Q**. how to implement tools?<br>
A. 
- by adding `@mcp.tool()` decorator to the functions
- by writing good docstrings. docstrings let llm know what this tool can do (tool usage), what is input and output for this tool.


example of mcp tool:
```python
@mcp.tool()
async def get_alerts(state: str) -> str:
    """Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    """
    url = f"{NWS_API_BASE}/alerts/active/area/{state}"
    data = await make_nws_request(url)

    if not data or "features" not in data:
        return "Unable to fetch alerts or no alerts found."

    if not data["features"]:
        return "No active alerts for this state."

    alerts = [format_alert(feature) for feature in data["features"]]
    return "\n---\n".join(alerts)
```


**Q.** what is decorator? <br>
A. In Python, a decorator is a special function that modifies the behavior of another function or method without changing its source code.

example:

In [2]:
def decorator_function(func):
    def wrapper_function(*args, **kwargs):
        # Code before calling the original function
        print(f"Wrapper executed before {func.__name__}")
        result = func(*args, **kwargs)
        # Code after calling the original function
        print(f"Wrapper executed after {func.__name__}")
        return result
    return wrapper_function


@decorator_function
def say_hello():
    print("Hello!")

say_hello()

Wrapper executed before say_hello
Hello!
Wrapper executed after say_hello


**Q**. how to run the server? <br>
A. here I add it to claude desktop and edit `claude_desktop_config.json` to introduce my mcp server:

```python
{
  "mcpServers": {
    "weather": {
      "command": "uv",
      "args": [
        "--directory",
        "/ABSOLUTE/PATH/TO/PARENT/FOLDER/weather",
        "run",
        "weather.py"
      ]
    }
  }
}
```